# Форматирование данных в Parquet

In [1]:
%reset -f

Форматирование файла с вакансиями 

In [15]:
%%bash
wc -l data/raw/vacancy_9.csv

132295 data/raw/vacancy_9.csv


In [ ]:
import pandas as pd
from tqdm import tqdm

lines_per_batch = 20000
total_rows = 132294
vacancy_df = pd.DataFrame()
part = 1

for chunk in tqdm(pd.read_csv(
    'data/raw/vacancy_9.csv',
    chunksize=lines_per_batch,
    sep="|",
    on_bad_lines='skip',
    low_memory=False,
    dtype=object
  ),
  total=total_rows // lines_per_batch + 1):
  vacancy_df = pd.concat([vacancy_df, chunk])

vacancy_df


100%|██████████| 7/7 [00:04<00:00,  1.67it/s]


,id,stateRegionCode,vacancyName,codeProfession,codeProfessionalSphere,professionalSphereName,vacancyAddress,vacancyAddressHouse,vacancyAddressAdditionalInfo,geo,...,hireDate,workPlace,medicalCertificate,scheduleTypeComment,benefitDetails,trainingDays,shift,medicalDocument,benefit,conditions
0,1dc5e1d0-f9b1-11ef-9ae2-25abf1fceba2,3600000000000,Продавец-кассир,NaN,Sales,"Продажи, закупки, снабжение, торговля","Воронежская область, Город Воронеж, Березовая ...",NaN,"Россия, г Воронеж, ул Березовая роща, д 4а","{""latitude"": 51.703059, ""longitude"": 39.227057}",...,NaN,"{""workPlaceForeign"": false, ""workPlaceOrdinary...",NaN,NaN,NaN,0,[],NaN,NaN,NaN
1,1d99efd0-f9b1-11ef-9ae2-25abf1fceba2,7600000000000,Продавец-кассир,NaN,Sales,"Продажи, закупки, снабжение, торговля","Ярославская область, Город Ярославль, Куропатк...",NaN,"Россия, г Ярославль, ул Куропаткова, зд 70","{""latitude"": 57.721108, ""longitude"": 39.771068}",...,NaN,"{""workPlaceForeign"": false, ""workPlaceOrdinary...",NaN,NaN,NaN,0,[],NaN,NaN,NaN
2,1d3b7c20-f9b1-11ef-9ae2-25abf1fceba2,5700000000000,Продавец-кассир,NaN,Sales,"Продажи, закупки, снабжение, торговля","Орловская область, Город Орёл, Половецкая улица",NaN,"Россия, г Орёл, ул Половецкая, д 18б","{""latitude"": 52.937543, ""longitude"": 36.088613}",...,NaN,"{""workPlaceForeign"": false, ""workPlaceOrdinary...",NaN,NaN,NaN,0,[],NaN,NaN,NaN
3,1cbb7c10-f9c4-11ef-acb2-d549be31d974,3600000000000,"Главный архивист отдела информации, публикации...",201931,StateServices,"Государственная служба, некоммерческие организ...","Воронежская область, Город Воронеж, Орджоникид...",дом: Д.31;,NaN,"{""latitude"": 51.65954, ""longitude"": 39.204671}",...,NaN,"{""workPlaceDisabilityGroup"": ""Функции эндокрин...",NaN,NaN,NaN,0,[],NaN,NaN,Допустимые
4,1c8376c0-f9b1-11ef-9ae2-25abf1fceba2,7600000000000,Продавец-кассир,NaN,Sales,"Продажи, закупки, снабжение, торговля","Ярославская область, Город Ярославль, Авиаторо...",NaN,"г Ярославль, пр-кт Авиаторов, д 20","{""latitude"": 57.650175, ""longitude"": 39.929926}",...,NaN,"{""workPlaceForeign"": false, ""workPlaceOrdinary...",NaN,NaN,NaN,0,[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132265,fd496d85-0f67-11ee-b544-0d972cff014f,6200000000000,Уборщик территории,192624,Industry,Производство,"Рязанская область, Город Рязань, Семинарская у...",дом 32,"ост. ""Дом художника""","{""latitude"": 54.633286, ""longitude"": 39.726655}",...,NaN,"{""workPlaceForeign"": false, ""workPlaceOrdinary...",NaN,NaN,NaN,0,[],NaN,NaN,Оптимальные
132266,fd489630-eec7-11ef-b3b9-cb26dff57dd7,7100000000000,Слесарь по ремонту дорожно-строительных машин ...,185226,BuldindRealty,"Строительство, ремонт, стройматериалы, недвижи...","Тульская область, Плавский район, Город Плавск...",20,"Тульская область, г. Плавск, ул. Островского, ...","{""latitude"": 53.703326, ""longitude"": 37.314939}",...,NaN,"{""workPlaceForeign"": false, ""workPlaceOrdinary...",NaN,NaN,NaN,0,[],NaN,NaN,Вредные
132267,fd47fc80-1630-11f0-82b1-d549be31d974,6800000000000,Рабочий по комплексному обслуживанию и ремонту...,175440,WorkingSpecialties,Рабочие специальности,"Тамбовская область, Город Тамбов, Советская ул...",дом: Д.33/16;,NaN,"{""latitude"": 52.710047, ""longitude"": 41.465793}",...,NaN,"{""workPlaceForeign"": false, ""workPlaceOrdinary...",NaN,NaN,NaN,0,[],Медицинская книжка,NaN,Допустимые
132268,fd400845-fcf2-11ec-a958-d768660431c4,3200000000000,печатник глубокой печати,165487,Forest,"Лесная, деревообрабатывающая, целлюлозно-бумаж...","Брянская область, Дятьковский район, Город Дят...",225 к1,NaN,"{""latitude"": 53.609172, ""longitude"": 34.335272}",...,NaN,"{""workPlaceForeign"": false, ""workPlaceOrdinary...",NaN,NaN,NaN,0,[],NaN,NaN,Вредные


In [87]:
vacancy_df.to_parquet('data/processed/vacancies.parquet', compression='zstd', engine='pyarrow')

Форматирование файла с резюме

In [82]:
%%bash
wc -l data/raw/cv.csv

10378731 data/raw/cv.csv


In [2]:
import pandas as pd
from tqdm import tqdm

lines_per_batch = 20000
total_rows = 10378730
cv_df = pd.DataFrame()

part = 1
for chunk in tqdm(pd.read_csv(
    'data/raw/cv.csv',
    chunksize=lines_per_batch,
    sep="|",
    on_bad_lines='skip',
    low_memory=False,
    dtype=object
  ),
  total=total_rows // lines_per_batch + 1):
  cv_df = pd.concat([cv_df, chunk])

  if (len(cv_df) > 1_000_000):
    cv_df.to_parquet(f'data/processed/cv.{part}.parquet', compression='zstd', engine='pyarrow')
    part += 1
    cv_df = pd.DataFrame()

cv_df.describe()


100%|██████████| 519/519 [06:13<00:00,  1.39it/s]


,id,candidateId,stateRegionCode,locality,localityName,birthday,gender,age,positionName,dateCreate,...,languageKnowledge,relocation,innerInfo,salaryMin,salaryMax,shifts,businessActivities,certifyingDocuments,additionalInformation,education
count,176867,176867,176861,176861,176811,176513,176867,176513,176863,176867,...,176866,176866,176866,176536,176535,176866,176865,176865,1784,126523
unique,176867,161976,90,18748,18691,20715,2,76,27572,170005,...,2247,3,165929,1653,1652,17,1995,1330,1660,10
top,fdab4180-40c8-11ee-9eec-01e20877b34e,cfd77ba0-bd14-11e6-a7f9-037acc02728d,7800000000000,7800000000000,г-Санкт-Петербург,2010-06-08T16:00:00+0400,Женский,15,Подсобный-рабочий,2025-04-07T08:12:23+0300,...,[],Не-готов-к-переезду,"{""rfCitizen"": ""Соискатель является гражданином...",30000,30000,[],[],[],не-имеет,Среднее-профессиональное
freq,1,41,7498,6675,6672,61,100989,11704,8188,9,...,124277,172376,43,17715,17716,170593,174404,175465,11,33056


In [3]:
cv_df.to_parquet(f'data/processed/cv.{part}.parquet', compression='zstd', engine='pyarrow')
cv_df = pd.DataFrame()

In [1]:
import dask.dataframe as dd

df = dd.read_parquet('data/processed/cv.*.parquet')

In [2]:
df.head()

,id,candidateId,stateRegionCode,locality,localityName,birthday,gender,age,positionName,dateCreate,...,languageKnowledge,relocation,innerInfo,salaryMin,salaryMax,shifts,businessActivities,certifyingDocuments,additionalInformation,education
0,1989bea0-c207-11ec-978d-1b29d3b53cbb,3314db80-c206-11ec-b47a-af9a27d731ed,4000000000000,4001800100000,"Калужская-область,-Мосальский-район,-г-Мосальск",1979-12-17T00:00:00+0300,Женский,45,Подсобный-рабочий,2022-04-22T09:40:29+0300,...,"[{""codeLanguage"": ""Русский"", ""level"": ""C2 — в ...",Не-готов-к-переезду,"{""idUser"": ""98544ff0-7fb2-11ea-86e1-dbfd80b8ec...",20000,20000,[],[],[],<NA>,Среднее
1,1989bea0-a67a-11ea-ad85-bfd13399602c,9fc9f2c0-a678-11ea-ad85-bfd13399602c,2100000000000,2100000100000,"Чувашская-Республика---чувашия,-г-Чебоксары",2000-04-10T00:00:00+0400,Мужской,25,Строитель,2020-06-04T18:43:13+0300,...,[],Не-готов-к-переезду,"{""idUser"": ""f21f0da0-1ff8-11e5-8442-1ff7059456...",15000,15000,[],[],[],<NA>,<NA>
2,1989be50-9130-11ef-8814-716c765c9fe3,d5f89dc0-8bb0-11ef-a669-192f67b3a632,7600000000000,7600000100000,"Ярославская-область,-г-Ярославль",2005-05-19T16:00:00+0400,Мужской,19,Тестировщик-ПО,2024-10-23T14:15:24+0300,...,"[{""codeLanguage"": ""Английский"", ""level"": ""A1 —...",Не-готов-к-переезду,"{""rfCitizen"": ""Соискатель является гражданином...",35000,35000,[],[],[],<NA>,Среднее-общее
3,1989be50-0125-11ef-84ca-716c765c9fe3,c5d02e20-0124-11ef-88e1-8f5fb6980b78,0400000000000,0400200000100,"Алтай-республика,-Кош-Агачский-район,-Кош-Агач...",2009-08-11T16:00:00+0400,Мужской,15,подсобный-рабочий,2024-04-23T06:53:52+0300,...,[],Не-готов-к-переезду,"{""idUser"": ""74ff8ef0-262c-11e5-b38b-1ff7059456...",15000,15000,[],[],[],<NA>,<NA>
4,1989bdb0-eb28-11ed-a58c-e57b87a63fa7,c39b8e70-d848-11ed-ac99-3315e3c40be6,2300000000000,2300000100000,"Краснодарский-край,-г-Краснодар",2007-01-18T15:00:00+0300,Женский,18,Подсобный-рабочий,2023-05-05T12:34:57+0300,...,"[{""codeLanguage"": ""Русский"", ""level"": ""A1 — на...",Не-готов-к-переезду,"{""idUser"": ""3c00ff60-90ff-11ea-8aed-5d9e90ab95...",4000,4000,[],[],[],<NA>,Среднее
